In [ ]:
import re
from pathlib import Path
import pysubs2

def ass_to_renpy(input_file, precision=0.005):
    subs = pysubs2.load(input_file, encoding="utf-8")

    output_lines = []

    last_speakers = []

    for line in subs:
        text_raw = re.sub(r'{.*?}', '', line.text).strip()

        # 检查是否是 scene 指令
        if text_raw.lower().startswith("scene"):
            output_lines.append(f"    {text_raw}")
            output_lines.append(f"    with dissolve\n")
            continue

        speaker = line.name.strip() if line.name else "narrator"
        text = text_raw

        start_time = round(line.start / 1000, 3)  # 毫秒转秒，保留三位小数
        end_time = round(line.end / 1000, 3)

        # 计算nw时间：字幕行时长
        duration = round(end_time - start_time, 3)

        # 处理多说话人情况
        speakers = speaker.split('_')
        num_speakers = len(speakers)
        if num_speakers == 1:
            dummy_tag = 'single'
        elif num_speakers == 2:
            dummy_tag = 'double'
        else:
            dummy_tag = 'all'

        has_image_change = speakers != last_speakers

        if has_image_change:
            # hide 上一组角色
            for spk in last_speakers:
                output_lines.append(f"    hide {spk}\n")
            # show 当前这一组角色
            for spk in speakers:
                output_lines.append(f"    show {spk} at {spk}_{dummy_tag}\n")
            last_speakers = speakers

        # 同步到字幕时间点
        output_lines.append(f"    while (renpy.music.get_pos() or 0) < {start_time}:")
        output_lines.append(f"        $ renpy.pause({precision}, hard=True)\n")

        # 正常输出字幕，增加fast和nw，改为公式形式
        if has_image_change:
            nw_expression = f"[{duration}-shorten-load_shorten]"
        else:
            nw_expression = f"[{duration}-shorten]"

        output_lines.append(
            f"    {speaker} \"{{font=j}}{{b}}{text}{{/b}}{{/font}}\\n\n"
            f"    {{font=z}}{{/font}}\\n\n"
            f"    {{fast}}{{nw={nw_expression}}}\"\n")

    # 结束后补充10秒 idle等待
    output_lines.append(f"    while (renpy.music.get_pos() or 0) < {start_time + 10.000}:")
    output_lines.append(f"        $ renpy.pause({precision}, hard=True)\n")
    output_lines.append(f"    # show idle\n")
    output_lines.append(
        f"    idle \"{{font=j}}{{b}}==idle=={{/b}}{{/font}}\\n\n"
        f"    {{font=z}}{{/font}}\\n\n"
        f"    {{fast}}{{nw=10.000}}\"\n")

    return output_lines

# 使用方法
if __name__ == "__main_3_":
    output_lines = ass_to_renpy("sub4renpy.ass")
    for line in output_lines:
        print(line, end='')

    # 确认后写入文件
    with open("output_script.rpy", "w", encoding="utf-8") as f:
        f.writelines(output_lines)


In [ ]:
output_lines = ass_to_renpy("sub4renpy.ass")
print('\n'.join(output_lines))

In [ ]:
import colorsys

def hex_to_rgb(hex_color):
    """把#rrggbb格式转换成RGB元组"""
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb_color):
    """把RGB元组转换成#rrggbb格式"""
    return '#{:02x}{:02x}{:02x}'.format(*rgb_color)

def average_hsv_color_hex(hex1, hex2):
    # 转为 RGB
    rgb1 = hex_to_rgb(hex1)
    rgb2 = hex_to_rgb(hex2)

    # 归一化 RGB 到 [0, 1]
    r1, g1, b1 = [x / 255.0 for x in rgb1]
    r2, g2, b2 = [x / 255.0 for x in rgb2]

    # 转 HSV
    h1, s1, v1 = colorsys.rgb_to_hsv(r1, g1, b1)
    h2, s2, v2 = colorsys.rgb_to_hsv(r2, g2, b2)

    # 处理 Hue 环形平均
    h1_deg = h1 * 360
    h2_deg = h2 * 360
    diff = abs(h1_deg - h2_deg)
    if diff > 180:
        if h1_deg > h2_deg:
            h2_deg += 360
        else:
            h1_deg += 360
    h_avg_deg = (h1_deg + h2_deg) / 2 % 360
    h_avg = h_avg_deg / 360

    # 线性平均 S 和 V
    s_avg = (s1 + s2) / 2
    v_avg = (v1 + v2) / 2

    # HSV 转回 RGB
    r_avg, g_avg, b_avg = colorsys.hsv_to_rgb(h_avg, s_avg, v_avg)

    # 放大到 0-255
    r_out = int(round(r_avg * 255))
    g_out = int(round(g_avg * 255))
    b_out = int(round(b_avg * 255))

    return rgb_to_hex((r_out, g_out, b_out))

# 示例：
hex1 = '#c042d5'  # 红色
hex2 = '#daf8c1'  # 蓝色

middle_hex = average_hsv_color_hex(hex1, hex2)
print(middle_hex)
